In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time

In [ ]:
driver = webdriver.Chrome()

source_url = "https://timkiem.vnexpress.net/?search_f=title,tag_list&q=OpenAI&media_type=all&fromdate=0&todate=0&latest=&cate_code=&date_format=month&"

driver.get(source_url)

source_platform = driver.find_element(By.CSS_SELECTOR, "a.logo").get_attribute("title")
print("Source platform:", source_platform)

search_words = driver.find_element(By.ID, "search_q").get_attribute("value")
print("Search words:", search_words)

select = Select(driver.find_element(By.ID, "thoigian"))
time_filter = select.first_selected_option
print("Time filter:", time_filter.text)

print("-"*40)

wait = WebDriverWait(driver, 10)
articles = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "article.item-news")))

for article in articles:
    title = article.find_elements(By.CSS_SELECTOR, 'h3.title-news')
    description = article.find_elements(By.CSS_SELECTOR, 'p.description')
    print(f'Title: {title[0].text}' if title else 'No title found')
    print(f'Description: {description[0].text}' if description else 'No description found')
    print('---------------------------------')


link_elements = driver.find_elements(By.CSS_SELECTOR, "h3.title-news a")
links = [el.get_attribute("href") for el in link_elements if "eclick" not in el.get_attribute("href")]

for link in links:
    driver.get(link)

    date_elements = driver.find_elements(By.CSS_SELECTOR, "span.date")
    print("Create_at:", date_elements[0].text if date_elements else "No date found")

    wait = WebDriverWait(driver, 50)

    total_comment = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "label#total_comment")))

    print("Total comments:", total_comment.text.strip() if total_comment else "No comment found")

    # Click tất cả nút "Xem thêm ý kiến" để load thêm comment
    while True:
        try:
            show_more_btn = driver.find_element(By.ID, "show_more_coment")
            if show_more_btn.is_displayed():
                driver.execute_script("arguments[0].click();", show_more_btn)
                time.sleep(1)
            else:
                break
        except:
            break

    # Click tất cả nút "Đọc tiếp" để hiển thị full content của comment dài
    read_more_btns = driver.find_elements(By.CSS_SELECTOR, "a.icon_show_full_comment.continue-reading")
    for btn in read_more_btns:
        try:
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.2)
        except:
            continue

    comments = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.content-comment")))

    if comments:
        for cmt in comments:
            try:
                # Kiểm tra xem có p.content_more hiển thị không (comment dài)
                content_more_els = cmt.find_elements(By.CSS_SELECTOR, "p.content_more")
                content = ""
                
                if content_more_els and content_more_els[0].is_displayed():
                    # Comment dài - lấy từ content_more, loại bỏ span.txt-name
                    content = driver.execute_script("""
                        var el = arguments[0];
                        var clone = el.cloneNode(true);
                        var txtName = clone.querySelector('span.txt-name');
                        if (txtName) txtName.remove();
                        return clone.textContent.trim();
                    """, content_more_els[0])
                else:
                    # Comment ngắn - lấy từ full_content, loại bỏ span.txt-name
                    full_content_els = cmt.find_elements(By.CSS_SELECTOR, "p.full_content")
                    if full_content_els:
                        content = driver.execute_script("""
                            var el = arguments[0];
                            var clone = el.cloneNode(true);
                            var txtName = clone.querySelector('span.txt-name');
                            if (txtName) txtName.remove();
                            return clone.textContent.trim();
                        """, full_content_els[0])
                
                if content:
                    print(content)
                    print("-" * 40)
            except Exception as e:
                continue      
    else:
        print("No comments found")
    
    print("=" * 80)
   
driver.quit()